In [ ]:
import argparse
import json
import os

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from nuscenes import NuScenes
from nuscenes.eval.prediction.config import load_prediction_config
from nuscenes.eval.prediction.splits import get_prediction_challenge_split
from nuscenes.prediction import PredictHelper
from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer
from nuscenes.prediction.input_representation.agents import AgentBoxesWithFadedHistory
from nuscenes.prediction.input_representation.interface import InputRepresentation
from nuscenes.prediction.input_representation.combinators import Rasterizer

In [ ]:
def get_im_and_box(nusc, token, cam_name="CAM_FRONT"):
    """
    Plots the camera view with bounding box around the agent to be predicted
    :param ax: matplotlib axes object
    :param nusc: NuScenes object
    :param token: Prediction token consisting of instance_sample token pair
    :param cam_name: Name of camera to use
    :return: (image, box, camera_instrinsic)
    """
    instance_token, sample_token = token.split('_')
    # get tokens
    sample = nusc.get('sample', sample_token)
    sample_cam_token = nusc.get('sample_data', sample['data'][cam_name])['token']
    annotations_in_sample = [nusc.get('sample_annotation', annotation) for annotation in sample['anns']]
    annotation_instance, *_ = (ann for ann in annotations_in_sample if ann['instance_token'] == instance_token)
    # plot
    data_path, boxes, camera_intrinsic = nusc.get_sample_data(sample_cam_token, selected_anntokens=[annotation_instance['token']])
    im = Image.open(data_path)
    if len(boxes) > 0:
        return im, boxes[0], camera_intrinsic
    else:
        return im, None, camera_intrinsic

def plot_cam_view(ax, nusc, token, cam_name='CAM_FRONT'):
    """
    Plots the camera view with bounding box around the agent to be predicted
    :param ax: matplotlib axes object
    :param nusc: NuScenes object
    :param token: Prediction token consisting of instance_sample token pair
    :param cam_name: Name of camera to use
    """
    im, box, camera_intrinsic = get_im_and_box(nusc, token, cam_name)
    ax.imshow(im)
    ax.set_title(cam_name)
    ax.axis('off')
    ax.set_aspect('equal')
    if box is not None:
        c = np.array(nusc.explorer.get_color(box.name)) / 255.0
        box.render(ax, view=camera_intrinsic, normalize=True, colors=(c, c, c))

In [ ]:
version = 'v1.0-mini'
data_root = '/Users/Gerry/Downloads/v1.0-mini'
split_name = 'mini_train'
config_name = 'predict_2020_icra.json'

In [ ]:
nusc = NuScenes(version=version, dataroot=data_root)
helper = PredictHelper(nusc)
dataset = get_prediction_challenge_split(split_name, dataroot=data_root)
config = load_prediction_config(helper, config_name)

In [ ]:
static_layer_rasterizer = StaticLayerRasterizer(helper)
agent_rasterizer = AgentBoxesWithFadedHistory(helper, seconds_of_history=3)
mtp_input_representation = InputRepresentation(static_layer_rasterizer, agent_rasterizer, Rasterizer())

In [ ]:
index = 0

In [ ]:
for token in dataset[index:index+1]:
    fig, axes = plt.subplots(1, 3, figsize=(18, 9))
    print(token)
    instance_token, sample_token = token.split('_')

    plot_cam_view(axes[1], nusc, token)
    plot_cam_view(axes[2], nusc, token, cam_name='CAM_FRONT_RIGHT')
    axes[0].imshow(mtp_input_representation.make_input_representation(instance_token, sample_token))
plt.show()
index += 2